In [7]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Path to the folder in your Google Drive
folder_path = '/content/drive/My Drive/Space Architecture/'


# List all files in the folder
files = os.listdir(folder_path)

# Print the list of files
print("Files in the folder:")
for file in files:
    print(file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in the folder:
schaenzlin-et-al-2006-structural-design-of-a-lunar-habitat.pdf
19770014162_update.pdf
Space Settlements - A Design Study (1977).pdf
system thinking 2024feb6.pptx
Stakeholders 2022feb6.pdf
Moon_Village_v1.1_PublicCDFStudy.pdf
Engineering, Design and Construction of Lunar Bases ASCE paper .pdf
Internal Pressure Level at the Lunar Base - Anna Petropoulos .pdf
The Lunar Base Handbook'99 Chapter 16 pp. 513-539 and Chapter 17 pp. 545-560.pdf
Habitats, Laboratories, and Airlocks pp.267-301.pdf
Chapter 7 Lunar Base Sitting pp.205-223.pdf
Chapter 13 pp.379-404.pdf
Chapter 13 pp.420-422.pdf
Chapter 5 pp. 114-151.pdf
Space Settlements a Design Study NASA 1977 Chapter 4. Choosing Among Alternatives pp. 41-61.pdf
Space Settlements a Design Study NASA 1977 chapter 3 Human needa in space pp. 21-36 .pdf


In [13]:
"first attempt to read in, now need to deal with image captioning but ran out of RAM"

'first attempt to read in, now need to deal with image captioning but ran out of RAM'

In [15]:
!pip install pdfplumber openai python-magic pymupdf pytesseract



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 28.1 MB/s eta 0:00:00


In [91]:
import pdfplumber
import matplotlib.pyplot as plt
from IPython.display import display
import re

def extract_sections_and_images_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = []
        images = []

        for page_num in range(10):  # Process only the first 10 pages for demonstration
            page = pdf.pages[page_num]

            left_rectangle = (10, 0, page.width / 2, page.height-50)
            left_text = page.crop(left_rectangle).extract_text()

            right_rectangle = (page.width / 2, 0, page.width, page.height)
            right_text = page.crop(right_rectangle).extract_text()

            pages.append(left_text+right_text)

            page_images = page.images

            descriptions={}
            for img in page_images:
                image_bbox = (img['x0'], page.height - img['y1'], img['x1'], page.height - img['y0'])
                caption_bbox = (max(0,img['x0']-50), page.height - img['y1'], img['x1'], min(792,page.height - img['y1'] + 500))
                caption_text = page.crop(caption_bbox).extract_text()
                caption_regex = r"Fig\.\s*\d+\..*"
                caption = None
                for line in caption_text.split("\n"):
                    match = re.match(caption_regex, line)
                    if match:
                        caption = match.group(0)
                        break

                cropped_page = page.crop(image_bbox)
                image_obj = cropped_page.to_image(resolution=400)
                images.append((image_obj, caption))


            # for img, title in images:
            #     # Placeholder for generative model integration to get description
            #     description = "Generated description for " + title  # Example placeholder
            #     descriptions[title] = description

            for page in pages:
              for title, description in descriptions.items():
                if title in page:
                    page = page.replace(title, title + " GPT GENERATED: " + description)

    for img, title in images:
        display(img)
        print("Title:", title)
        print("------------")

    return pages, images

pdf_path = "/content/drive/My Drive/Space Architecture/schaenzlin-et-al-2006-structural-design-of-a-lunar-habitat.pdf"

pages, images = extract_sections_and_images_from_pdf(pdf_path)

# Print the sections
for i, page in enumerate(pages, start=1):
    print(f"Page {i}:", page)


TypeError: can only concatenate str (not "NoneType") to str

In [89]:
for page_image,title in images:
  PILIMAGE=page_image.original
  print(PILIMAGE)

<PIL.Image.Image image mode=RGB size=250x250 at 0x7E0AC6C58160>
<PIL.Image.Image image mode=RGB size=1299x902 at 0x7E0ABDD51000>
<PIL.Image.Image image mode=RGB size=1999x1127 at 0x7E0AC4C08EB0>
<PIL.Image.Image image mode=RGB size=1099x775 at 0x7E0AC8015F00>
<PIL.Image.Image image mode=RGB size=1333x896 at 0x7E0AC80169E0>
<PIL.Image.Image image mode=RGB size=1099x608 at 0x7E0AC8015F90>
<PIL.Image.Image image mode=RGB size=1106x507 at 0x7E0AC49E86D0>
<PIL.Image.Image image mode=RGB size=1099x809 at 0x7E0AC8015060>
<PIL.Image.Image image mode=RGB size=999x549 at 0x7E0AC49E8E50>
<PIL.Image.Image image mode=RGB size=1319x910 at 0x7E0AC816E3B0>
<PIL.Image.Image image mode=RGB size=1334x753 at 0x7E0AC49E8EE0>
<PIL.Image.Image image mode=RGB size=1286x1274 at 0x7E0AC816E8F0>


In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration

page_image,title = images[0]
image=page_image.original

model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

rule_id="61fc849b203dd1124d0ec122"
# Image_style="color"
# image_path = f"selectedRules/{rule_id}_{Image_style}.png"
# image = Image.open(image_path).convert("RGB")

prompt = "<image>\nWhat does the image depict that is related to it's title "+title+"\nASSISTANT:"


# prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
# url = "https://www.ilankelman.org/stopsigns/australia.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt")

# Generate
generate_ids = model.generate(**inputs, max_length=100)
print(processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])



config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]